# LangSmith

In [2]:
import openai
import os 
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import gradio as gr

from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base= os.getenv('OPENAI_API_BASE')
openai.api_type= "azure"
openai.api_version = os.getenv('OPENAI_API_VERSION')
deployment:str=os.getenv('CHATGPT_MODEL')
embeddings_deployment:str=os.getenv('EMBEDDINGS_MODEL')

llm = AzureChatOpenAI(
    deployment_name=deployment,
    model_name=deployment
    )


In [4]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_PROJECT'] = 'langsmith-chat-demo'


In [6]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory
)


In [7]:
with gr.Blocks() as chat_system:
    chat = gr.Chatbot()
    prompt = gr.Textbox(placeholder="What's on your mind?")
    clear = gr.ClearButton([prompt, chat])
    clear.click(conversation.memory.clear)

    def llm_reply(prompt, chat_history):
        reply = conversation.predict(input=prompt)
        chat_history.append((prompt, reply))
        return "", chat_history

    prompt.submit(llm_reply, [prompt, chat], [prompt, chat])

chat_system.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
